# 집과 관련된 데이터들을 통해 집값을 예측을합니다.

# Import Module

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor


import matplotlib.pyplot as plt
import seaborn as sns

### PCA변환 ###
from sklearn.decomposition import PCA





In [2]:
# train_data_path = join('../input/2019-2nd-ml-month-with-kakr', 'train.csv')
train_data_path = './input/train.csv'
# sub_data_path = join('../input/2019-2nd-ml-month-with-kakr', 'test.csv')
sub_data_path = './input/test.csv'

## 1. 데이터 살펴보기
pandas의 read_csv 함수를 사용해 데이터를 읽어오고, 각 변수들이 나타내는 의미를 살펴보겠습니다.
1. ID : 집을 구분하는 번호
2. date : 집을 구매한 날짜
3. price : 타겟 변수인 집의 가격
4. bedrooms : 침실의 수
5. bathrooms : 침실당 화장실 개수
6. sqft_living : 주거 공간의 평방 피트
7. sqft_lot : 부지의 평방 피트
8. floors : 집의 층 수
9. waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
10. view : 집이 얼마나 좋아 보이는지의 정도
11. condition : 집의 전반적인 상태
12. grade : King County grading 시스템 기준으로 매긴 집의 등급
13. sqft_above : 지하실을 제외한 평방 피트
14. sqft_basement : 지하실의 평방 피트
15. yr_built : 집을 지은 년도
16. yr_renovated : 집을 재건축한 년도
17. zipcode : 우편번호
18. lat : 위도
19. long : 경도
20. sqft_living15 : 2015년 기준 주거 공간의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)
21. sqft_lot15 : 2015년 기준 부지의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)

In [3]:
data = pd.read_csv(train_data_path)
data_train = pd.read_csv(train_data_path)
data_test = pd.read_csv(sub_data_path)
print('train data dim : {}'.format(data.shape))
print('sub data dim : {}'.format(data_test.shape))

train data dim : (15035, 21)
sub data dim : (6468, 20)


In [4]:
data_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [5]:
data_train['date'] = data_train['date'].apply(lambda i: i[:6]).astype(int)
data_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,201410,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,201502,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,201502,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,201406,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,201501,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [6]:
test_id = data_test['id']

In [7]:

y = data_train['price']

del data_train['price']
print(data_train.columns)

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [8]:
del data_train['id']

print(data_train.columns)

Index(['date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


### 치우친 값에 대해 Log를 취하지만 예측을 위해 사용하지 않습니다.

In [9]:
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

# for c in skew_columns:
#     data_train[c] = np.log1p(data_train[c].values)

In [10]:
y = np.log1p(y)

In [11]:
data_test['date'] = data_test['date'].apply(lambda i: i[:6]).astype(int)

del data_test['id']

print(data_test.columns)

Index(['date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [12]:
train_len = len(data_train)
data_train = pd.concat((data_train, data_test), axis=0)

In [13]:
data_train.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,201410,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,201502,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,201502,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,201406,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,201501,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


## 2. 간단한 전처리 

In [14]:
test = data_test.iloc[train_len:, :]
x = data_train.iloc[:train_len, :]
print(y.shape)
print(x.shape)
y1 = y.iloc[:train_len]

(15035,)
(15035, 19)


## 3. 모델링
### Average Blending
여러가지 모델의 결과를 산술 평균을 통해 Blending 모델을 만들겠습니다.

### Grid Search
그리드 서치를 통해 파라미터를 찾습니다.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [16]:
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

In [17]:
gboost = GradientBoostingRegressor(random_state=2019)
xgboost1 = xgb.XGBRegressor(random_state=2019)
xgboost2 = xgb.XGBRegressor(random_state=2019)
xgboost3 = xgb.XGBRegressor(random_state=2019)
lightgbm = lgb.LGBMRegressor(random_state=2019)
rdforest = RandomForestRegressor(random_state=2019)
# models = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'},
#           {'model':lightgbm, 'name':'LightGBM'},{'model':rdforest,'name':'RandomForest'}]
models = [{'model':xgboost1, 'name':'XGBRegressor1'},{'model':xgboost2,'name':'XGBRegressor2'},{'model':xgboost3,'name':"XGBRegressor3"}]

In [18]:
def get_scores(models, train, y):
    df = {}
    
    for model in models:
        model_name = model.__class__.__name__
        print(model["name"])
        X_train, X_test, y_train, y_test = train_test_split(train, y, random_state=2019, test_size=0.2)
        model["model"].fit(X_train, y_train)
        
        y_pred = model["model"].predict(X_test)        
        df[model_name] = rmse(y_test, y_pred)
        score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)
        print(score_df)
    return score_df

get_scores(models, x, y)

XGBRegressor1
               RMSE
dict  142352.990035
XGBRegressor2
               RMSE
dict  142352.990035
XGBRegressor3
               RMSE
dict  142352.990035


,RMSE
dict,142352.990035


In [19]:
def my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5):
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', \
                              cv=5, verbose=verbose, n_jobs=n_jobs)
    
    # 모델 fitting
    grid_model.fit(train, y)

    # 결과값 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임 생성
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE 값 계산 후 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')
    
    return results

In [20]:
from sklearn.model_selection import GridSearchCV
lgb_param_grid = {
    'n_estimators': [50,60,70,80,90, 100],
    'max_depth': [1,2,3,4,5,6,7,8,9, 10],
    'learning_rate' : [0.05],
    'num_iterations' : [1000] ,   
}
rf_param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
xgb_param_grid = {        
        'gamma': [0.5, 2, 5],        
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [6, 7, 8],
        'n_estimators' : [100]
        }


In [31]:
# for model in models:    
# my_GridSearch(lightgbm,x,y,lgb_param_grid)
# my_GridSearch(rdforest,x,y,rf_param_grid)
my_GridSearch(xgboost,x,y,xgb_param_grid)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed: 12.2min
[Parallel(n_jobs=5)]: Done 135 out of 135 | elapsed: 52.7min finished


,colsample_bytree,gamma,max_depth,n_estimators,score,RMSLE
2,0.6,0.5,8,100,-0.028810,0.169736
1,0.6,0.5,7,100,-0.029041,0.170413
11,0.8,0.5,8,100,-0.029148,0.170729
10,0.8,0.5,7,100,-0.029239,0.170994
0,0.6,0.5,6,100,-0.029377,0.171396
9,0.8,0.5,6,100,-0.029537,0.171864
20,1.0,0.5,8,100,-0.030785,0.175458
19,1.0,0.5,7,100,-0.030897,0.175775
18,1.0,0.5,6,100,-0.031702,0.178052
4,0.6,2.0,7,100,-0.034121,0.184720


### Cross Validation
교차 검증을 통해 모델의 성능을 간단히 평가하겠습니다.

In [21]:
lightgbm1 = lgb.LGBMRegressor(n_estimator = 200, max_depth = 9, learning_rate = 0.05, num_iterations = 1000)
rdforest1 = RandomForestRegressor(random_state=2019, max_depth = 100, max_features = 3, min_samples_leaf = 3,min_samples_split = 9, n_estimators = 200 )
xgboost1 = xgb.XGBRegressor(booster='gbtree',max_depth=8, n_estimators=200,learning_rate=0.02,num_iterations = 1000, random_state=2019,alpha = 0.8, gamma = 0.5, colsample_bytree = 0.6)
xgboost2 = xgb.XGBRegressor(booster='gbtree',max_depth=7, n_estimators=200,learning_rate=0.015,num_iterations = 1000, random_state=2019,alpha = 0.3, gamma = 0.5)
xgboost3 = xgb.XGBRegressor(booster='gbtree',max_depth=8, n_estimators=200,learning_rate=0.03,num_iterations = 1000, random_state=2019,alpha = 1.5, gamma = 0.5)

In [22]:
x_train, x_valid, y_train, y_valid = train_test_split(x.values, y.values, test_size=0.2, shuffle=False, random_state=34)
print(x_train.shape)
print(y_train.shape)
print(data_test.values.shape)
eval_set=[(x_valid, y_valid)]
xgboost1.fit(x_train, y_train, eval_set=eval_set, eval_metric = "rmse",verbose = False, early_stopping_rounds=50)
lightgbm1.fit(x_train, y_train, eval_set=eval_set, eval_metric = "rmse",verbose = False, early_stopping_rounds=50)
xgboost2.fit(x_train, y_train, eval_set=eval_set, eval_metric = "rmse",verbose = False, early_stopping_rounds=50)
xgboost3.fit(x_train, y_train, eval_set=eval_set, eval_metric = "rmse",verbose = False, early_stopping_rounds=50)
# rdforest1.fit(x_train, y_train, eval_set=eval_set, eval_metric = "rmse",verbose = False, early_stopping_rounds=50)
search_models = [{'model':xgboost1, 'name':'XGBRegressor1'},{'model':xgboost2,'name':'XGBRegressor2'},{'model':xgboost3,'name':'XGBRegressor3'}]
# search_models = [{'model':serach_lgb, 'name':'LightGBM'},{'model':search_rf,'name':'RandomForest'}]

(12028, 19)
(12028,)
(6468, 19)
[01:17:05] WARNING: ../src/learner.cc:541: 
Parameters: { num_iterations } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[01:17:08] WARNING: ../src/learner.cc:541: 
Parameters: { num_iterations } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:17:09] WARNING: ../src/learner.cc:541: 
Parameters: { num_iterations } might not be used.

  This may not be accur

In [23]:
def get_cv_score(models):
    kfold = KFold(n_splits=5, random_state=2019).get_n_splits(x)
    for m in models:
        print("Model {} CV score : {:.4f}".format(m['name'], np.mean(cross_val_score(m['model'], x, y)), 
                                                 kf=kfold))
#     print("Model {} CV score : {:.4f}".format(models, np.mean(cross_val_score(models, x, y)), 
#                                              kf=kfold))

In [24]:
get_cv_score(search_models)

[01:17:15] WARNING: ../src/learner.cc:541: 
Parameters: { num_iterations } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:17:16] WARNING: ../src/learner.cc:541: 
Parameters: { num_iterations } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:17:17] WARNING: ../src/learner.cc:541: 
Parameters: { num_iterations } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


### Make Submission

회귀 모델의 경우에는 cross_val_score 함수가 R<sup>2</sup>를 반환합니다.<br>
R<sup>2</sup> 값이 1에 가까울수록 모델이 데이터를 잘 표현함을 나타냅니다. 3개 트리 모델이 상당히 훈련 데이터에 대해 괜찮은 성능을 보여주고 있습니다.<br> 훈련 데이터셋으로 3개 모델을 학습시키고, Average Blending을 통해 제출 결과를 만들겠습니다.

In [25]:
def AveragingBlending(models, x, y, sub_x):
    
    for m in models : 
        m['model'].fit(x, y)
    
    sub_x = sub_x[f_names]
    predictions = np.column_stack([
        m['model'].predict(sub_x.values) for m in models
    ])
    return np.mean(predictions, axis=1)

In [26]:

test_pred1 = np.expm1(xgboost1.predict(data_test.values))
test_pred2 = np.expm1(xgboost2.predict(data_test.values))
test_pred3 = np.expm1(xgboost3.predict(data_test.values))
test_pred4 = np.expm1(lightgbm1.predict(data_test.values))
y_pred = (test_pred1 + test_pred2 + test_pred3) /3 

In [27]:
sub = pd.DataFrame(data={'id':test_id,'price':y_pred})
print(sub.shape)
sub.head

(6468, 2)


<bound method NDFrame.head of          id         price
0     15035  4.082227e+05
1     15036  2.967213e+05
2     15037  1.007189e+06
3     15038  2.421112e+05
4     15039  2.342760e+05
...     ...           ...
6463  21498  1.858515e+05
6464  21499  3.336667e+05
6465  21500  3.468493e+05
6466  21501  2.736845e+05
6467  21502  3.347004e+05

[6468 rows x 2 columns]>

In [28]:
sub.to_csv('submission1.csv', index=False)

![nn](./my_score.png)

### 생각
- 아무리해도 20만점 밑으로 내려가지 않는다.
- 데이터 전처리를 좀더 정확하게 하면 좋을것 같지만 아직은 아이디어가 떠오르지 않는다.
- 너무 어렵다.